# O Algoritmo KMeans

## 1. Introdução

Esta é uma implementação do algoritmo KMeans usada para ilustrar seu funcionamento interno.

Neste exemplo, apenas temos a intenção de explicar o algoritmo. Para uso em produção, sugerimos usar uma implementação mais madura como o [sklearn.cluster.KMeans][1].

O artigo [Wikipedia KMeans article][2] tem uma boa explicação da matemática do algoritmo KMeans, bem como variações e referências para maiores detalhes. 


  [1]: https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html
  [2]: https://en.wikipedia.org/wiki/K-means_clustering

## 2. Importação dos dados 

Os dados usados usados nesta demonstração são de casos e mortes por covid19 por município dos Estados Unidos. 

In [ ]:
def bloco():
    
    global df
    
    from datetime import datetime
    import pandas as pd
    import os
    pd.options.display.max_columns = None
        
    if os.path.isfile('/kaggle/input/us-counties-covid-19-dataset/us-counties.csv'):
        file = '/kaggle/input/us-counties-covid-19-dataset/us-counties.csv'
    elif os.path.isfile( 'data/us-counties-covid-19-dataset.csv' ):
        file = 'data/us-counties-covid-19-dataset.csv'
    else:
        raise Exception('Por favor, inclua o arquivo de dados em data/us-counties-covid-19-dataset.csv')
        
    df = pd.read_csv( file,
             #nrows=1000 
        )

    df.insert( 1, "wday", df['date'].apply( lambda x: int(datetime.strptime(x,'%Y-%m-%d').strftime('%w')) ) )
    df.insert( 1, "mday", df['date'].apply( lambda x: int(datetime.strptime(x,'%Y-%m-%d').strftime('%d')) ) )
    #df.insert( 1, "month", df['date'].apply( lambda x: int(datetime.strptime(x,'%Y-%m-%d').strftime('%m')) ) )

    df = df.dropna()
    df = df.sample( 50000 )
    
    #df['wday'] = (df['wday'] - df['wday'].mean())/df['wday'].std()
    #df['cases'] = (df['cases'] - df['cases'].mean())/df['cases'].std()
    #df['deaths'] = (df['deaths'] - df['deaths'].mean())/df['deaths'].std()
    
    return df
    
bloco()

## 3. O Algoritmo

A classe abaixo contém uma implementação comentada do algoritmo que, em resumo faz:
* Mapeia as colunas do dataset de entrada como eixos de um espaço n-dimensional.
* Define "k" centroides, inicialmente retirando com a coordenada de pontos sorteados aleatóriamente;
* Refina os centroids:
 * Associa todos os pontos ao centroide mais próximo usando distância euclidiana;
 * Recaulcula a posição do centroid como a média das posições dos pontos associados;
* Executa este refinamento até que:
  * Nenhum centroid tenha se movido mais que a distância "stop" 
  * Não tenha sido alcançado o limite máximo de execuções "max_iter"

Existe uma exceção quando os dois ou mais centroides acabam muito próximos. Neste caso pode acontecer de nenhum ponto ser associado ao centroide. Neste caso um ponto aleatório será sorteado novamente para subsitiuí-lo. 

In [ ]:
import random
import math
import numpy as np
import pandas as pd
from scipy.spatial import distance 
        
def fit_kmeans( X, stop=0.2, max_iter=20, k=3, verbose=True, on_step=None ):
    """Find kmeans clusters for the input dataset
    
    Parameters
    ----------
    X : list of points
        array of points to find the clusters in (accept: list, numpy or pandas)
    stop : float
        stop processing if the clusters move less than this distance
    max_iter : int
        stop processing after this number of iteractions
    k : int
        number of clusters to find
    verbose : bool
        print some messages during processing
    on_step : function name
        callback function executed on every iteration of the model with this signature:
    
        def abc( step, centroids, centroid_x_points, centroid_x_avgdist ):
            pass           
    
    """
    
    
    _centroids = []
    _centroid_x_points = []
    _centroid_x_avgdist = []

    if isinstance(X, pd.DataFrame):
        X = X.to_numpy()

    X = list(X)
        
    _centroids = []
    for centroid_number in range(k):
        _centroids.append( random.sample( X, 1 )[0] )        

    iter = 0
    change = stop+1
    while True:        

        # preserva os centroides antigos para permitir o cálculo de distância
        centroids_old = list(_centroids)

        if verbose: print( f'centroid={_centroids}' )     
        if verbose: print( f'iteração={iter}' )     

        ##############################################################
        # PARTE 1:
        # Associa os pontos aos centroides e calcula a distância média
        ##############################################################

        # inicializa variáveis 
        _centroid_x_points = []
        _centroid_x_avgdist = []
        for centroid_number in range(k):
            _centroid_x_points.append([])
            _centroid_x_avgdist.append(0)

        for x in X:                

            # calcula a distância para do ponto x para cada centroide
            distances = []            
            for centroid in _centroids:           
                distances.append( distance.euclidean( centroid, x ) )
                           # a distância euclideana é basicamente pitágoras 
                           # (quadrado da hipotenusa = soma dos quadadros dos catetos)
                           # leia-se: sqtr((x1 - x2)^2 + (y1 - y2)^2 (z1 - z2)^2 (...))

            # associa o ponto ao centroide mais próximo
            centroid_number = np.argmin( distances )
            _centroid_x_points[ centroid_number ].append( x )  

            # totaliza as distâncias do ponto x ao centroid
            _centroid_x_avgdist[ centroid_number ] = _centroid_x_avgdist[ centroid_number ] + distances[ centroid_number ]

        # cálculo da distância média
        for centroid_number in range(k):
            if len(  _centroid_x_points[ centroid_number ] ) == 0:
                _centroid_x_avgdist[ centroid_number ] = math.nan
            else:
                _centroid_x_avgdist[ centroid_number ] = _centroid_x_avgdist[ centroid_number ] / len( _centroid_x_points[ centroid_number ] )


        if not( on_step is None ):
            on_step( iter, _centroids, _centroid_x_points, _centroid_x_avgdist )

        ##############################
        # PARTE 2 
        # Condições de Saída
        ##############################

        # Na iteração anterior nenhum centroid se moveu mais que a distância stop 
        if change < stop:
            return iter, _centroids, _centroid_x_points, _centroid_x_avgdist                   

        # Não foi alcançado o limite máximo de iterações
        if( iter > max_iter ):
            return iter, _centroids, _centroid_x_points, _centroid_x_avgdist

        ######################################################
        # PARTE 3 
        # Move o centroide para o centro dos pontos associados
        ######################################################
        iter = iter + 1
        change = 0.0                
        # para cada centroid calcula a nova posição 
        for centroid_number in range(k):            
            points = _centroid_x_points[ centroid_number ]

            if len(points) > 0:
                new_center = np.average( points, axis=0 )
            else:
                new_center = random.sample( X, 1 )[0]

            if verbose: print( f' n={centroid_number} points={len(points)} c={new_center}' )

            _centroids[ centroid_number ] = new_center

            change = max( 
                distance.euclidean( 
                    _centroids[centroid_number], 
                    centroids_old[centroid_number]
                ), 
                change 
            )

            

## 4. Exemplo de agrupamentos casos x mortes

No exemplo abaixo, procuramos agrupamentos entre casos e mortes.

A animação abaixo ilustra como os pontos vão se reposicionando. Em cada iteração o modelo encontra os pontos mais próximos do centroide e reposiciona o centroide no centro dos pontos selecionados. Isto faz o modelo espalhar os pontos.


In [ ]:
def bloco():
    
    global model
    
    import matplotlib.pyplot as plt
    import matplotlib.animation 
    import matplotlib.image as mpimg
    
    from IPython import display

    colors_centroid = ['#800000','#008000','#000080']
    colors_point = ['#FF000080','#00FF0080','#0000FF80']
   
    fig = plt.figure(figsize=(10,10))
    
    anim = matplotlib.animation.PillowWriter(fps=1)
    anim.setup(fig,"tmp.gif",dpi=150)    
    
        
    def draw_frame( step, _centroids,_centroid_x_points,_centroid_x_avgdist ):      
        plt.clf()
        ax = fig.add_subplot(1, 1, 1)                      
        
        ax.set_xlabel('cases')
        ax.set_ylabel('deaths')
        plt.title( f"Exemplo KMeans (iteracao={step}; k=3)")
        
        for centroid_number, centroid in enumerate(_centroids):            
            s1 = ax.scatter(        
                [ a[0] for a in _centroid_x_points[centroid_number] ],
                [ a[1] for a in _centroid_x_points[centroid_number] ],
                marker='x', s=30,
                color=colors_point[centroid_number],
                label = f'centroid {centroid_number} points'
            )   

        for centroid_number, centroid in enumerate(_centroids):
            s2 = ax.scatter( 
                centroid[0], 
                centroid[1], 
                marker='o', s=100,
                color=colors_centroid[centroid_number],                
                label = f'centroid {centroid_number}'
            )
            
        ax.legend()
        anim.grab_frame()    
        display.clear_output(wait=True)
        print( '### PROCESSANDO ###' )
        display.display(fig)    
        
       
   
    (step, _centroids,_centroid_x_points,_centroid_x_avgdist) = fit_kmeans( 
            df[ [ 'cases','deaths' ] ], 
            k=3, 
            max_iter=15, 
            verbose=False, 
            on_step=draw_frame 
        ) 
   
    for _ in range(5):
        anim.grab_frame()    
        
    anim.finish()
    plt.close()
    display.clear_output(wait=True)
    
    import base64
    with open('tmp.gif', 'rb') as fd:
        b64 = base64.b64encode(fd.read()).decode('ascii')
        display.display( display.HTML(f'<img src="data:image/gif;base64,{b64}" width=800/>') )
        
    print( '' )
    print( 'Centroides:' )
    print( _centroids )
    print( '' )
    print( 'Distâncias médias dos pontos aos centroides:')
    print( _centroid_x_avgdist )

   
    
bloco()

## 5. Exemplo de agrupamentos casos x mortes x dia semana 

No exemplo abaixo, procuramos agrupamentos entre casos, mortes e dia da semana.


In [ ]:
def bloco():
    
    global model
    
    import matplotlib.pyplot as plt
    import matplotlib.animation 
    import matplotlib.image as mpimg
    
    from IPython import display

    colors_centroid = ['#800000','#008000','#000080','#808000','#800080','#008080']
    colors_point = ['#FF000080','#00FF0080','#0000FF80','#FFFF0080','#FF00FF80','#00FFFF80']
   
    fig = plt.figure(figsize=(15,15))
    
    anim = matplotlib.animation.PillowWriter(fps=1)
    anim.setup(fig,"tmp.gif",dpi=96)    
    
        
    def draw_frame( step, _centroids,_centroid_x_points,_centroid_x_avgdist ):      
        plt.clf()
        ax = fig.add_subplot(1, 1, 1, projection='3d')       
    
        plt.title( f"Exemplo KMeans (iteracao={step}; k=6)")
        
        ax.set_xlabel('cases')
        ax.set_ylabel('deaths')
        ax.set_zlabel('wday')

        for centroid_number, centroid in enumerate(_centroids):
            ax.scatter(        
                [ a[0] for a in _centroid_x_points[centroid_number] ],
                [ a[1] for a in _centroid_x_points[centroid_number] ],
                [ a[2] for a in _centroid_x_points[centroid_number] ],
                marker='x', s=30,
                color=colors_point[centroid_number]
            )

        for centroid_number, centroid in enumerate(_centroids):
            ax.scatter( 
                centroid[0], 
                centroid[1], 
                centroid[2],
                marker='o', s=100,
                color=colors_centroid[centroid_number] 
            )  
            
        anim.grab_frame()            
        display.clear_output(wait=True)
        print( '### PROCESSANDO ITERACAO=' + str(step) + ' ###' )
        display.display(fig)       
        
     
    
    df['cases_pad'] = ( df['cases'] - df['cases'].mean() ) / df['cases'].std()
    df['deaths_pad' ] = ( df['deaths'] - df['deaths'].mean() ) / df['deaths'].std()
    df['wday_pad'] = ( df['wday'] - df['wday'].mean() ) / df['wday'].std()
   
    (step, _centroids,_centroid_x_points,_centroid_x_avgdist) = fit_kmeans( 
        df[ [ 'cases_pad', 'deaths_pad', 'wday_pad' ] ], 
        k=6, 
        max_iter=15, 
        verbose=False, 
        on_step=draw_frame 
    )    
   
    for _ in range(5):
        anim.grab_frame()    
        
    anim.finish()
    plt.close()
    display.clear_output(wait=True)
    
    import base64
    with open('tmp.gif', 'rb') as fd:
        b64 = base64.b64encode(fd.read()).decode('ascii')
        display.display( display.HTML(f'<img src="data:image/gif;base64,{b64}" width=800 />') )

    print( '' )
    print( 'The centroids:' )
    print( _centroids )
    print( '' )
    print( 'Average distances:')
    print( _centroid_x_avgdist )
    
bloco()

In [ ]:
import os
os.remove("tmp.gif")